<a href="https://colab.research.google.com/github/maruchinatan/FacilitaEats/blob/main/FacilitaEats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
%pip -q install google-genai

In [33]:
!pip install PyPDF2

In [34]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [35]:
!pip install -q google-adk

In [106]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from google.adk.tools import google_search
from datetime import date
import textwrap
# Removido HTML, IPython.display é usado apenas para Markdown no final.
# Se não estiver em um ambiente Jupyter/Colab, display e Markdown podem precisar de alternativas.
try:
    from IPython.display import display, Markdown
except ImportError:
    def Markdown(text): # Fallback simples se IPython não estiver disponível
        return text
    def display(obj): # Fallback simples
        print(obj)

import requests
from bs4 import BeautifulSoup
import re
import PyPDF2
import warnings
import sys
import json # Importado para lidar com JSON

warnings.filterwarnings("ignore")


In [97]:
# Funções Auxiliares (Comuns a Todos os Agentes)
def call_agent(agent: Agent, message_text: str) -> str:
    """Função para chamar um agente e obter a resposta."""
    session_service = InMemorySessionService()
    session = session_service.create_session(
        app_name=agent.name, user_id="user1", session_id="session1"
    )
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text
                    # Removido o \n extra aqui, pois o LLM geralmente já formata.
                    # Se precisar, pode adicionar de volta.
    return final_response.strip() # .strip() para remover espaços extras no início/fim

def to_markdown(text):
    """Função para formatar texto para exibição em Markdown (pode precisar de IPython.display)."""
    text = text.replace("•", "  *")
    return Markdown(textwrap.indent(text, "> ", predicate=lambda _: True))


In [107]:
########################################### --- Agente 1: busca_restaurante --- ###########################################
def agente_busca_restaurante():
    """Define o agente para buscar restaurantes."""

    busca_restaurante = Agent(
        name="busca_restaurante",
        # ATENÇÃO: Verifique se "gemini-1.5-flash-latest" é o modelo correto e disponível
        # para o google.adk. Pode ser necessário usar "gemini-2.0-flash" se for o caso,
        # ou outro modelo compatível com o ADK.
        model="gemini-1.5-flash-latest",
        instruction="""
            Você é um assistente de busca de restaurantes.
            Com base nas preferências do usuário (localização, tipo de comida,
            restrições alimentares, ingredientes-chave),
            você deve usar a ferramenta Google Search para encontrar até 5 restaurantes
            que atendam aos critérios na cidade especificada.
            Priorize restaurantes bem avaliados no Google Maps.

            Sua resposta DEVE ser uma string JSON válida contendo uma lista de objetos.
            Cada objeto deve representar um restaurante e ter as chaves "title" (nome do restaurante)
            e "link" (link para o Google Maps ou site do restaurante).
            Se informações como endereço ("address") ou avaliação ("rating") estiverem
            disponíveis diretamente nos resultados da ferramenta Google Search, inclua-as também.

            Exemplo de formato de resposta:
            [
              {"title": "Restaurante Exemplo 1", "link": "https://maps.google.com/?cid=12345", "address": "Rua Exemplo, 123", "rating": "4.5 estrelas"},
              {"title": "Restaurante Exemplo 2", "link": "https://maps.google.com/?cid=67890", "address": "Av. Teste, 456", "rating": "4.2 estrelas"}
            ]

            Se nenhum resultado for encontrado pela ferramenta Google Search ou se os resultados
            não puderem ser formatados corretamente, retorne uma lista JSON vazia: [].
            Não adicione nenhum texto explicativo, comentários ou marcadores de markdown como ```json antes ou depois da string JSON.
            Retorne APENAS a string JSON.
        """,
        tools=[google_search],
    )
    return busca_restaurante

def formatar_restaurantes(lista_restaurantes_parseada):
    """Formata os resultados da busca (já parseados como lista de dicts) para exibição."""

    if not lista_restaurantes_parseada:
        return "Nenhum restaurante encontrado com esses critérios.", []

    mensagem = "Aqui estão alguns restaurantes que correspondem às suas preferências:\n\n"
    restaurantes_formatados_lista = [] # Renomeado para clareza
    for i, resultado_dict in enumerate(lista_restaurantes_parseada):
        nome = resultado_dict.get('title', 'Nome não encontrado')
        link = resultado_dict.get('link', 'Link não encontrado')
        address = resultado_dict.get('address')
        rating = resultado_dict.get('rating')

        mensagem += f"[{i + 1}] {nome}\n"
        if address:
            mensagem += f"   Endereço: {address}\n"
        if rating:
            mensagem += f"   Avaliação: {rating}\n"
        mensagem += f"   Link: {link}\n\n"
        restaurantes_formatados_lista.append({'nome': nome, 'link': link, 'address': address, 'rating': rating})
    return mensagem, restaurantes_formatados_lista

In [100]:
########################################### --- Agente 2: facilita_pedido --- ###########################################
def agente_facilita_pedido():
    """Define o agente para facilitar a escolha de pratos."""
    facilita_pedido = Agent(
        name="facilita_pedido",
        model="gemini-1.5-flash-latest", # Verifique o nome do modelo
        instruction="""
            Você é um assistente de escolha de pratos em restaurantes.
            Você receberá o texto de um cardápio e as preferências do usuário.
            Analise o cardápio e sugira até 3 pratos que atendam aos critérios.
            Forneça o nome, descrição (se disponível no cardápio) e preço (se disponível) de cada prato.
            Se não encontrar pratos adequados, informe.
            Formate sua resposta de forma clara e concisa.
        """,
        # tools=[Google Search], # Removido, pois a análise é sobre texto fornecido.
                                 # Adicione se o agente precisar buscar algo externamente.
    )
    return facilita_pedido

def extrair_cardapio(url_restaurante):
    """
    Extrai o cardápio do restaurante.
    ATENÇÃO: Esta função é um placeholder e precisará de MUITA melhoria
    para lidar com a variedade de sites de restaurantes e formatos de cardápio.
    A URL recebida pode ser do Google Maps, exigindo primeiro encontrar o site real.
    """
    print(f"DEBUG: Tentando extrair cardápio de {url_restaurante}")
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
        response = requests.get(url_restaurante, headers=headers, timeout=10)
        response.raise_for_status()
        content_type = response.headers.get("Content-Type", "").lower()

        # Se a URL for do Google Maps, o conteúdo não será o cardápio diretamente.
        # Esta é uma grande simplificação e provavelmente falhará para URLs do Google Maps.
        if "html" in content_type:
            print("DEBUG: Extraindo como HTML.")
            return extrair_cardapio_de_html(response.text)
        elif "pdf" in content_type:
            print("DEBUG: Extraindo como PDF.")
            # Para extrair PDF de URL, você já tem a lógica que salva e lê.
            # Vamos simplificar aqui, assumindo que extrair_cardapio_de_pdf pode lidar com response.content
            # with open("cardapio_temp.pdf", "wb") as f:
            #     f.write(response.content)
            # return extrair_cardapio_de_pdf_local("cardapio_temp.pdf")
            # A sua função original `extrair_cardapio_de_pdf` já espera uma URL, então está ok.
            return extrair_cardapio_de_pdf(url_restaurante) # Sua função original
        elif "text/plain" in content_type:
            print("DEBUG: Extraindo como texto.")
            return response.text
        else:
            print(f"DEBUG: Content-Type não suportado diretamente: {content_type}. Tentando HTML como fallback.")
            # Se o tipo não for claro, mas for texto, tentar HTML pode funcionar às vezes.
            if response.text:
                 return extrair_cardapio_de_html(response.text)
            return "Não foi possível determinar o tipo de conteúdo do cardápio ou conteúdo vazio."

    except requests.exceptions.Timeout:
        return f"Erro ao acessar o cardápio: Timeout para {url_restaurante}"
    except requests.exceptions.RequestException as e:
        return f"Erro ao acessar o cardápio: {e}"

def extrair_cardapio_de_html(html_cardapio):
    """Extrai o texto de uma página HTML. Simplificado."""
    soup = BeautifulSoup(html_cardapio, "html.parser")
    # Remove scripts e styles para limpar o texto
    for script_or_style in soup(["script", "style"]):
        script_or_style.decompose()
    # Obtém o texto, separando por espaços e removendo excesso de novas linhas
    text = ' '.join(soup.stripped_strings)
    return text if text else "Conteúdo HTML não resultou em texto extraível."


def extrair_cardapio_de_pdf(url_pdf): # Nome original da sua função
    """Extrai o texto de um cardápio em PDF a partir de uma URL."""
    try:
        headers = {'User-Agent': 'Mozilla/5.0'} # Simples user-agent
        response = requests.get(url_pdf, headers=headers, timeout=10)
        response.raise_for_status()

        # Salva temporariamente o PDF para leitura
        temp_pdf_filename = "temp_cardapio.pdf"
        with open(temp_pdf_filename, "wb") as f:
            f.write(response.content)

        text = ""
        with open(temp_pdf_filename, "rb") as pdf_file:
            reader = PyPDF2.PdfReader(pdf_file)
            for page_num in range(len(reader.pages)):
                page_text = reader.pages[page_num].extract_text()
                if page_text:
                    text += page_text + "\n"

        # os.remove(temp_pdf_filename) # Opcional: remover o arquivo temp depois de usar
        return text if text else "PDF não continha texto extraível ou estava vazio."
    except requests.exceptions.Timeout:
        return f"Erro ao acessar o PDF: Timeout para {url_pdf}"
    except requests.exceptions.RequestException as e:
        return f"Erro ao baixar o PDF: {e}"
    except PyPDF2.errors.PdfReadError as e:
        # Isso pode acontecer se o arquivo não for um PDF válido ou estiver corrompido.
        # Tentar retornar o conteúdo bruto como texto pode ser um último recurso, mas improvável de ser útil.
        try:
            return response.text # Se a leitura do PDF falhar, talvez seja um HTML/TXT disfarçado.
        except:
            return f"Erro ao ler o PDF: {e}. O arquivo pode estar corrompido ou não ser um PDF padrão."
    except Exception as e:
        return f"Erro inesperado ao processar PDF: {e}"

def sugerir_pratos(cardapio_texto, preferencias_usuario_dict):
    """
    PLACEHOLDER: Função para simular a sugestão de pratos.
    No futuro, isso seria uma chamada ao agente_facilita_pedido com o texto do cardápio.
    """
    print("DEBUG: Chamando placeholder sugerir_pratos.")
    if not cardapio_texto or "Erro" in cardapio_texto or "Não foi possível" in cardapio_texto:
        return "Não foi possível obter o cardápio para sugerir pratos."

    # Simulação simples
    pratos_sugeridos = [
        {"nome": "Prato Exemplo 1 (do Cardápio Simulado)", "descricao": "Descrição com base nas preferências e cardápio", "preco": "R$ XX,XX"},
        {"nome": "Prato Exemplo 2 (do Cardápio Simulado)", "descricao": "Outra sugestão relevante", "preco": "R$ YY,YY"},
    ]
    return pratos_sugeridos



In [101]:
########################################### --- Agente 3: escolha_bebidas --- ###########################################
def agente_escolha_bebidas():
    """Define o agente para escolher bebidas."""
    escolha_bebidas = Agent(
        name="escolha_bebidas",
        model="gemini-1.5-flash-latest", # Verifique o nome do modelo
        instruction="""
            Você é um assistente de escolha de bebidas.
            Você receberá o texto de um cardápio.
            Analise o cardápio e apresente as opções de bebidas,
            idealmente organizadas por categorias (ex: Com Álcool - Cervejas, Vinhos, Coquetéis;
            Sem Álcool - Refrigerantes, Sucos, Águas).
            Se não encontrar bebidas ou não puder categorizar, apenas liste o que encontrar
            ou informe que não há opções claras de bebidas.
            Formate sua resposta de forma clara.
        """,
        # tools=[Google Search], # Removido
    )
    return escolha_bebidas

def organizar_bebidas(cardapio_texto):
    """PLACEHOLDER: Simula organização de bebidas."""
    print("DEBUG: Chamando placeholder organizar_bebidas.")
    if not cardapio_texto or "Erro" in cardapio_texto or "Não foi possível" in cardapio_texto:
        return "Não foi possível obter o cardápio para listar bebidas."
    # Simulação
    return {
        "Com Álcool (Exemplo)": ["Cerveja Simulada", "Vinho Simulado"],
        "Sem Álcool (Exemplo)": ["Refrigerante Simulado", "Suco Simulado"],
    }


In [102]:
########################################### --- Agente 4: escolha_sobremesa --- ###########################################
def agente_escolha_sobremesa():
    """Define o agente para escolher sobremesas."""
    escolha_sobremesa = Agent(
        name="escolha_sobremesa",
        model="gemini-1.5-flash-latest", # Verifique o nome do modelo
        instruction="""
            Você é um assistente de escolha de sobremesas.
            Você receberá o texto de um cardápio.
            Analise o cardápio e apresente as opções de sobremesa.
            Se não houver sobremesas disponíveis, informe isso claramente.
            Formate sua resposta de forma clara.
        """,
        # tools=[Google Search], # Removido
    )
    return escolha_sobremesa

def listar_sobremesas(cardapio_texto):
    """PLACEHOLDER: Simula listagem de sobremesas."""
    print("DEBUG: Chamando placeholder listar_sobremesas.")
    if not cardapio_texto or "Erro" in cardapio_texto or "Não foi possível" in cardapio_texto:
        return ["Não foi possível obter o cardápio para listar sobremesas."]
    # Simulação
    return ["Sobremesa Simulada A", "Torta Simulada B"]



In [104]:
########################################### --- Agente 5: finalizacao_pedido --- ###########################################
def agente_finalizacao_pedido():
    """Define o agente para finalizar o pedido e fornecer informações do restaurante."""
    finalizacao_pedido = Agent(
        name="finalizacao_pedido",
        model="gemini-1.5-flash-latest", # Verifique o nome do modelo
        instruction="""
            Você é um assistente de finalização de pedidos.
            Você receberá o nome de um restaurante e uma URL de referência (ex: Google Maps).
            Use a ferramenta Google Search, se necessário, para encontrar e fornecer
            informações úteis sobre este restaurante, como:
            - Endereço completo (confirmar ou obter).
            - Horário de funcionamento.
            - Telefone de contato.
            - Um link direto para o site oficial do restaurante ou para fazer uma reserva, se encontrar.
            Se alguma informação não for encontrada, apenas omita ou diga que não foi encontrada.
            Formate a resposta de forma clara e útil para o usuário.
        """,
        tools=[Google_Search], # Necessário para buscar informações adicionais
    )
    return finalizacao_pedido

def compilar_pedido(restaurante_nome, prato_nome, bebida_nome, sobremesa_nome):
    """Função para compilar o pedido do usuário."""
    pedido = {
        "restaurante": restaurante_nome,
        "prato": prato_nome,
        "bebida": bebida_nome,
        "sobremesa": sobremesa_nome,
    }
    return pedido

# def buscar_info_restaurante(nome_restaurante): # Esta função foi incorporada no agente_finalizacao_pedido
#     """Busca informações do restaurante (placeholder)."""
#     return "Informações do restaurante (a serem implementadas pelo agente_finalizacao_pedido)"




In [ ]:
########################################### --- Fluxo Principal do FacilitaEats --- ###########################################
if __name__ == "__main__":
    print("🍽️ Bem-vindo ao FacilitaEats! 🍽️")

    # --- Passo 1: Busca de Restaurante ---
    agente_busca = agente_busca_restaurante()
    preferencias_usuario = {
        "localizacao": input("Em qual cidade você está? (Ex: São Paulo): ").strip(),
        "tipo_comida": input("Que tipo de comida você prefere? (Ex: Italiana, Japonesa): ").strip(),
        "restricoes": input("Restrições alimentares? (Ex: Vegetariano, Vegano, Nenhuma): ").strip(),
        "ingredientes": input("Ingredientes-chave? (Ex: Frango, Cogumelos, Queijo): ").strip(),
    }
    mensagem_busca = f"""
        Encontre restaurantes com as seguintes características:
        Localização: {preferencias_usuario['localizacao']}
        Tipo de comida: {preferencias_usuario['tipo_comida']}
        Restrições: {preferencias_usuario['restricoes']}
        Ingredientes: {preferencias_usuario['ingredientes']}
    """
    print("\nBuscando restaurantes... Por favor, aguarde.\n")
    resultados_string_json = call_agent(agente_busca, mensagem_busca)

    lista_restaurantes_parseada = []
    if resultados_string_json:
        try:
            # Tenta extrair JSON de forma mais robusta (pode estar entre ```json ... ```)
            match = re.search(r'\[.*\]', resultados_string_json, re.DOTALL)
            json_str_to_parse = resultados_string_json # Default to the whole string
            if match:
                json_str_to_parse = match.group(0)

            # Verifica se a string realmente parece um JSON antes de tentar o parse
            if json_str_to_parse.strip().startswith('[') and json_str_to_parse.strip().endswith(']'):
                lista_restaurantes_parseada = json.loads(json_str_to_parse)
            else:
                print(f"AVISO: A resposta do agente não foi formatada como um JSON de lista esperado.")
                print(f"Resposta recebida:\n-----\n{resultados_string_json}\n-----")


        except json.JSONDecodeError as e:
            print(f"Erro ao decodificar JSON da resposta do agente: {e}")
            print(f"String JSON recebida para análise:\n-----\n{resultados_string_json}\n-----")
        except Exception as e: # Captura outras exceções durante o parse
            print(f"Erro inesperado ao processar resposta do agente: {e}")
            print(f"Resposta recebida:\n-----\n{resultados_string_json}\n-----")
    else:
        print("AVISO: O agente de busca de restaurantes não retornou nenhuma resposta.")


    print("\n--- 🔍 Resultados da Busca de Restaurantes ---\n")
    mensagem_formatada, lista_restaurantes_obj = formatar_restaurantes(lista_restaurantes_parseada)
    print(mensagem_formatada)

    # --- Usuário escolhe um restaurante ---
    restaurante_escolhido_dados = None # Dicionário com nome, link, etc.
    url_referencia_restaurante = "Nenhuma URL disponível"
    restaurante_escolhido_nome = "Nenhum restaurante disponível"

    if lista_restaurantes_obj:
        try:
            escolha_num_str = input(f"Digite o número do restaurante desejado (1 a {len(lista_restaurantes_obj)}), ou Enter para pular: ").strip()
            if not escolha_num_str:
                print("Nenhuma seleção feita. Continuando sem restaurante selecionado.")
            else:
                escolha_idx = int(escolha_num_str) - 1
                if 0 <= escolha_idx < len(lista_restaurantes_obj):
                    restaurante_escolhido_dados = lista_restaurantes_obj[escolha_idx]
                else:
                    print("Escolha fora do intervalo. Tente novamente mais tarde ou selecione o primeiro da próxima vez.")
        except ValueError:
            print("Entrada inválida. Nenhum restaurante selecionado.")

        if restaurante_escolhido_dados:
            restaurante_escolhido_nome = restaurante_escolhido_dados.get('nome', 'Nome não disponível')
            url_referencia_restaurante = restaurante_escolhido_dados.get('link', 'URL não disponível')
    else:
        print("Nenhum restaurante encontrado para fazer uma escolha.")

    print(f"\nRestaurante Escolhido: {restaurante_escolhido_nome}")
    if restaurante_escolhido_dados:
        print(f"URL de Referência: {url_referencia_restaurante}")

    # --- Extração do Cardápio (se um restaurante foi escolhido) ---
    cardapio_texto_extraido = ""
    if restaurante_escolhido_dados and url_referencia_restaurante != 'URL não disponível':
        print(f"\n--- 📄 Extraindo Cardápio de {restaurante_escolhido_nome} ---")
        print(f"Tentando acessar: {url_referencia_restaurante}")
        cardapio_texto_extraido = extrair_cardapio(url_referencia_restaurante)

        if not cardapio_texto_extraido or \
           "Erro ao acessar" in cardapio_texto_extraido or \
           "Não foi possível determinar" in cardapio_texto_extraido or \
           "Conteúdo HTML não resultou" in cardapio_texto_extraido or \
           "PDF não continha texto" in cardapio_texto_extraido:
            print(f"\nAVISO: Não foi possível extrair o cardápio de forma útil.")
            print(f"Detalhe: {cardapio_texto_extraido[:300]}...") # Mostra parte da resposta/erro
            cardapio_texto_extraido = "" # Garante que está vazio se a extração falhou
        else:
            print(f"Cardápio extraído com sucesso (primeiros 300 caracteres):\n{cardapio_texto_extraido[:300]}...")
    else:
        print("\nNenhum restaurante selecionado ou URL válida para extrair cardápio.")

    # --- Passos 2, 3, 4: Escolha de Prato, Bebida, Sobremesa (usando agentes e cardápio) ---
    prato_escolhido_final = "Não especificado"
    bebida_escolhida_final = "Não especificada"
    sobremesa_escolhida_final = "Não especificada"

    if cardapio_texto_extraido:
        print("\n--- 🍽️ Analisando cardápio para Pratos ---")
        # IMPLEMENTAÇÃO REAL: Chamar agente_facilita_pedido
        # agente_prato = agente_facilita_pedido()
        # mensagem_prato = f"Preferências: {preferencias_usuario}\n\nCardápio:\n{cardapio_texto_extraido[:7000]}" # Limitar tamanho
        # sugestoes_pratos_texto = call_agent(agente_prato, mensagem_prato)
        # print(sugestoes_pratos_texto)
        # prato_escolhido_final = input("Digite o prato escolhido (ou Enter para pular): ").strip() or prato_escolhido_final

        # Usando placeholder por enquanto:
        sugestoes_pratos_simulado = sugerir_pratos(cardapio_texto_extraido, preferencias_usuario)
        print("Sugestões (simuladas):", sugestoes_pratos_simulado)
        prato_escolhido_final = "Prato Simulado Escolhido" # Simulação

        print("\n--- 🍹 Analisando cardápio para Bebidas ---")
        # IMPLEMENTAÇÃO REAL: Chamar agente_escolha_bebidas
        # agente_bebida = agente_escolha_bebidas()
        # mensagem_bebida = f"Cardápio:\n{cardapio_texto_extraido[:7000]}"
        # opcoes_bebidas_texto = call_agent(agente_bebida, mensagem_bebida)
        # print(opcoes_bebidas_texto)
        # bebida_escolhida_final = input("Digite a bebida escolhida (ou Enter para pular): ").strip() or bebida_escolhida_final

        # Usando placeholder:
        opcoes_bebidas_simulado = organizar_bebidas(cardapio_texto_extraido)
        print("Opções de Bebidas (simuladas):", opcoes_bebidas_simulado)
        bebida_escolhida_final = "Bebida Simulada Escolhida" # Simulação

        print("\n--- 🍰 Analisando cardápio para Sobremesas ---")
        # IMPLEMENTAÇÃO REAL: Chamar agente_escolha_sobremesa
        # agente_sobremesa = agente_escolha_sobremesa()
        # mensagem_sobremesa = f"Cardápio:\n{cardapio_texto_extraido[:7000]}"
        # opcoes_sobremesas_texto = call_agent(agente_sobremesa, mensagem_sobremesa)
        # print(opcoes_sobremesas_texto)
        # sobremesa_escolhida_final = input("Digite a sobremesa escolhida (ou Enter para pular): ").strip() or sobremesa_escolhida_final

        # Usando placeholder:
        opcoes_sobremesas_simulado = listar_sobremesas(cardapio_texto_extraido)
        print("Opções de Sobremesas (simuladas):", opcoes_sobremesas_simulado)
        sobremesa_escolhida_final = "Sobremesa Simulada Escolhida" # Simulação
    else:
        print("\nComo o cardápio não pôde ser extraído ou analisado, pulando sugestões detalhadas.")

    # --- Passo 5: Finalização do Pedido ---
    pedido_compilado = compilar_pedido(
        restaurante_escolhido_nome, prato_escolhido_final, bebida_escolhida_final, sobremesa_escolhida_final
    )
    print("\n\n--- 📝 Resumo do Pedido Final ---\n")
    for item, valor in pedido_compilado.items():
        print(f"{item.capitalize()}: {valor}")

    if restaurante_escolhido_dados:
        agente_finalizacao = agente_finalizacao_pedido()
        print("\nBuscando informações finais do restaurante...")
        mensagem_finalizacao = f"""
        O usuário escolheu o restaurante: {restaurante_escolhido_nome}.
        A URL de referência para este restaurante é: {url_referencia_restaurante}.
        Por favor, usando a ferramenta Google Search se necessário, encontre e forneça:
        - Endereço completo (confirmar ou obter).
        - Horário de funcionamento.
        - Telefone de contato.
        - Um link direto para o site oficial do restaurante ou para fazer uma reserva, se encontrar.
        Se alguma informação não for encontrada, apenas omita ou diga que não foi encontrada.
        Formate a resposta de forma clara e útil para o usuário.
        """
        info_adicionais_restaurante = call_agent(agente_finalizacao, mensagem_finalizacao)
        print("\n--- ℹ️ Informações Adicionais do Restaurante ---\n")
        try:
            # Se estiver em ambiente que suporta Markdown (ex: Jupyter)
            display(to_markdown(info_adicionais_restaurante))
        except NameError: # Fallback se display ou to_markdown não estiverem definidos
            print(info_adicionais_restaurante)
    else:
        print("\nNenhum restaurante foi selecionado para buscar informações adicionais.")

    print("\n🎉 Obrigado por usar o FacilitaEats! 🎉")

🍽️ Bem-vindo ao FacilitaEats! 🍽️
